In [1]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

### Dataset

download kaggle dataset: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset/download?datasetVersionNumber=1  
download liar dataset: https://www.cs.ucsb.edu/~william/data/liar_dataset.zip

In [2]:
from load_dataset import kaggle_dataset
from load_dataset import liar_dataset
from load_dataset import kaggle_dataset_title

kaggle_path = "./dataset/Fake-News/"
liar_path = "./dataset/Liar/"

# Pick dataset you want:
# In any case the plain text is too long for training, 
# you can import kaggle_title as input
# -- If kaggle title:
dataset = kaggle_dataset_title(kaggle_path)
# -- If kaggle:
# dataset = kaggle_dataset(kaggle_path)
# -- If Liar:
# dataset = liar_dataset(liar_path)

train_sentences = dataset.train_sentences
val_sentences = dataset.val_sentences
train_labels = dataset.train_labels
val_labels = dataset.val_labels

### Model

##### Word Enbedding:  
pre-trained text embedding model from TensorFlow Hub  
more text-embedding download in https://tfhub.dev/s?module-type=text-embedding  
NNLM:  
plain text -> value [batch_size, 128]  
BERT:  
plain text -> pooled_output [batch_size, 128]  
              sequence_output [batch_size, seq_length, 128]  

##### Network

In [11]:
# Split a small batch of dataset to continue (save time...)
i = 3000
train_x = train_sentences[:i]
val_x = val_sentences[:int(0.2*i)]
train_y = train_labels[:i]
val_y = val_labels[:int(0.2*i)]

In [12]:
embedding = "nnlm"

# Select a model
from load_model import lstm
# lstm(LSTM_units, embedding)
from load_model import cnn_lstm
# cnn_lstm(cov_filters, cov_kernel, pool_size, LSTM_units, embedding)
from load_model import dense
# dense(embedding, neural=64)
from load_model import dense_dropout
# dense_dropout(embedding, dropout=0.1, neural_1=256, neural_2=64)

model = dense_dropout()
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_history = model.fit(train_x, train_y, validation_data=(val_x, val_y),epochs=5)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_5[0][0]']                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

### Text Attack
TextAttack is a Python framework for adversarial attacks, data augmentation, and model training in NLP.
https://github.com/QData/TextAttack

In [5]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper
from textattack import AttackArgs
from textattack import Attacker
from textattack.datasets import Dataset

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1-logits, logits), dim=1)
        return final_preds

##### unsuccessful attacks

In [6]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import BAEGarg2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 5 / 5:   5%|▌         | 5/100 [00:00<00:05, 18.67it/s]textattack: Saving checkpoint under "checkpoints\1670905110083.ta.chkpt" at 2022-12-12 23:18:30 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 3 / 7 / 10:  10%|█         | 10/100 [00:29<04:29,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1670905139784.ta.chkpt" at 2022-12-12 23:18:59 after 10 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 3 / 8 / 11:  11%|█         | 11/100 [00:29<04:02,  2.73s/it]

[Succeeded / Failed / Skipped / Total] 0 / 4 / 11 / 15:  15%|█▌        | 15/100 [00:44<04:14,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1670905154774.ta.chkpt" at 2022-12-12 23:19:14 after 15 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 4 / 13 / 17:  17%|█▋        | 17/100 [00:44<03:39,  2.65s/it]

...

[Succeeded / Failed / Skipped / Total] 0 / 42 / 43 / 85:  85%|████████▌ | 85/100 [06:50<01:12,  4.83s/it]textattack: Saving checkpoint under "checkpoints\1670905520755.ta.chkpt" at 2022-12-12 23:25:20 after 85 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 42 / 44 / 86:  86%|████████▌ | 86/100 [06:50<01:06,  4.78s/it]

[Succeeded / Failed / Skipped / Total] 0 / 46 / 44 / 90:  90%|█████████ | 90/100 [07:21<00:49,  4.91s/it]textattack: Saving checkpoint under "checkpoints\1670905551438.ta.chkpt" at 2022-12-12 23:25:51 after 90 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 49 / 46 / 95:  95%|█████████▌| 95/100 [07:49<00:24,  4.94s/it]textattack: Saving checkpoint under "checkpoints\1670905579405.ta.chkpt" at 2022-12-12 23:26:19 after 95 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 54 / 46 / 100: 100%|██████████| 100/100 [08:32<00:00,  5.12s/it]textattack: Saving checkpoint under "checkpoints\1670905621994.ta.chkpt" at 2022-12-12 23:27:01 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 54 / 46 / 100: 100%|██████████| 100/100 [08:32<00:00,  5.12s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 54     |
| Number of skipped attacks:    | 46     |
| Original accuracy:            | 54.0%  |
| Accuracy under attack:        | 54.0%  |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 13.25  |
| Avg num queries:              | 202.56 |
+-------------------------------+--------+


In [15]:
# Model: kaggle, dense_dropout, nnlm
from textattack.attack_recipes import Pruthi2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 10     
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  1
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:  10%|█         | 1/10 [3:27:32<31:07:54, 12452.73s/it]
[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:  50%|█████     | 5/10 [02:47<02:47, 33.45s/it]textattack: Saving checkpoint under "checkpoints\1670804432652.ta.chkpt" at 2022-12-11 19:20:32 after 5 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 8 / 2 / 10: 100%|██████████| 10/10 [24:20<00:00, 146.01s/it]textattack: Saving checkpoint under "checkpoints\1670805725468.ta.chkpt" at 2022-12-11 19:42:05 after 10 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 8 / 2 / 10: 100%|██████████| 10/10 [24:20<00:00, 146.01s/it]





+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 0       |
| Number of failed attacks:     | 8       |
| Number of skipped attacks:    | 2       |
| Original accuracy:            | 80.0%   |
| Accuracy under attack:        | 80.0%   |
| Attack success rate:          | 0.0%    |
| Average perturbed word %:     | nan%    |
| Average num. words per input: | 546.5   |
| Avg num queries:              | 10492.0 |
+-------------------------------+---------+


In [14]:
# Model: liar, dense_dropout, nnlm
from textattack.attack_recipes import IGAWang2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100     
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): ImprovedGeneticAlgorithm(
    (pop_size):  60
    (max_iters):  20
    (temp):  0.3
    (give_up_if_no_improvement):  False
    (post_crossover_check):  False
    (max_crossover_retries):  20
    (max_replace_times_per_index):  5
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_percent):  0.2
        (compare_against_original):  True
      )
    (1): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (max_mse_dist):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  False
      )
    (2): StopwordModification
  (is_black_box):  True
) 





[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:   1%|          | 1/100 [06:11<10:13:25, 371.78s/it]







textattack: Saving checkpoint under "checkpoints\1670806506597.ta.chkpt" at 2022-12-11 19:55:06 after 5 attacks.












textattack: Saving checkpoint under "checkpoints\1670806652575.ta.chkpt" at 2022-12-11 19:57:32 after 10 attacks.










textattack: Saving checkpoint under "checkpoints\1670806692369.ta.chkpt" at 2022-12-11 19:58:12 after 15 attacks.


...









textattack: Saving checkpoint under "checkpoints\1670807472728.ta.chkpt" at 2022-12-11 20:11:12 after 85 attacks.










textattack: Saving checkpoint under "checkpoints\1670807524401.ta.chkpt" at 2022-12-11 20:12:04 after 90 attacks.










textattack: Saving checkpoint under "checkpoints\1670807551651.ta.chkpt" at 2022-12-11 20:12:31 after 95 attacks.












textattack: Saving checkpoint under "checkpoints\1670807696744.ta.chkpt" at 2022-12-11 20:14:56 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [20:50<00:00, 12.50s/it]





+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 0       |
| Number of failed attacks:     | 57      |
| Number of skipped attacks:    | 43      |
| Original accuracy:            | 57.0%   |
| Accuracy under attack:        | 57.0%   |
| Attack success rate:          | 0.0%    |
| Average perturbed word %:     | nan%    |
| Average num. words per input: | 17.7    |
| Avg num queries:              | 3348.68 |
+-------------------------------+---------+


In [13]:
# Model: liar, dense_dropout, nnlm
from textattack.attack_recipes import CheckList2020 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapExtend
    (1): WordSwapContract
    (2): WordSwapChangeName
    (3): WordSwapChangeNumber
    (4): WordSwapChangeLocation
    )
  (constraints): 
    (0): RepeatModification
  (is_black_box):  True
) 











textattack: Saving checkpoint under "checkpoints\1670806301548.ta.chkpt" at 2022-12-11 19:51:41 after 5 attacks.










textattack: Saving checkpoint under "checkpoints\1670806311147.ta.chkpt" at 2022-12-11 19:51:51 after 10 attacks.









textattack: Saving checkpoint under "checkpoints\1670806314297.ta.chkpt" at 2022-12-11 19:51:54 after 15 attacks.


...







textattack: Saving checkpoint under "checkpoints\1670806358365.ta.chkpt" at 2022-12-11 19:52:38 after 85 attacks.









textattack: Saving checkpoint under "checkpoints\1670806360852.ta.chkpt" at 2022-12-11 19:52:40 after 90 attacks.








textattack: Saving checkpoint under "checkpoints\1670806362854.ta.chkpt" at 2022-12-11 19:52:42 after 95 attacks.








textattack: Saving checkpoint under "checkpoints\1670806366525.ta.chkpt" at 2022-12-11 19:52:46 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [01:09<00:00,  1.44it/s]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 57    |
| Number of skipped attacks:    | 43    |
| Original accuracy:            | 57.0% |
| Accuracy under attack:        | 57.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 17.7  |
| Avg num queries:              | 12.42 |
+-------------------------------+-------+


In [6]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import Pruthi2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100     
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=5,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.functional.Functional'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  False
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  False
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  False
      )
    (3): WordSwapQWERTY
    )
  (constraints): 
    (0): MaxWordsPerturbed(
        (max_num_words):  1
        (compare_against_original):  True
      )
    (1): MinWordLength
    (2): StopwordModification
    (3): RepeatModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 4 / 1 / 5:   5%|▌         | 5/100 [00:02<00:41,  2.32it/s]textattack: Saving checkpoint under "checkpoints\1670905949204.ta.chkpt" at 2022-12-12 23:32:29 after 5 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 4 / 3 / 7:   7%|▋         | 7/100 [00:02<00:28,  3.21it/s]

[Succeeded / Failed / Skipped / Total] 0 / 6 / 4 / 10:  10%|█         | 10/100 [00:03<00:30,  2.96it/s]textattack: Saving checkpoint under "checkpoints\1670905950427.ta.chkpt" at 2022-12-12 23:32:30 after 10 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 6 / 5 / 11:  11%|█         | 11/100 [00:03<00:27,  3.24it/s]

[Succeeded / Failed / Skipped / Total] 0 / 9 / 6 / 15:  15%|█▌        | 15/100 [00:04<00:26,  3.20it/s]textattack: Saving checkpoint under "checkpoints\1670905951728.ta.chkpt" at 2022-12-12 23:32:31 after 15 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 9 / 11 / 20:  20%|██        | 20/100 [00:04<00:18,  4.23it/s]textattack: Saving checkpoint under "checkpoints\1670905951774.ta.chkpt" at 2022-12-12 23:32:31 after 20 attacks.


...

[Succeeded / Failed / Skipped / Total] 0 / 50 / 35 / 85:  85%|████████▌ | 85/100 [00:22<00:04,  3.73it/s]textattack: Saving checkpoint under "checkpoints\1670905969804.ta.chkpt" at 2022-12-12 23:32:49 after 85 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 52 / 38 / 90:  90%|█████████ | 90/100 [00:23<00:02,  3.81it/s]textattack: Saving checkpoint under "checkpoints\1670905970653.ta.chkpt" at 2022-12-12 23:32:50 after 90 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 54 / 41 / 95:  95%|█████████▌| 95/100 [00:24<00:01,  3.89it/s]textattack: Saving checkpoint under "checkpoints\1670905971452.ta.chkpt" at 2022-12-12 23:32:51 after 95 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [00:25<00:00,  3.85it/s]textattack: Saving checkpoint under "checkpoints\1670905973028.ta.chkpt" at 2022-12-12 23:32:53 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [00:25<00:00,  3.85it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 57     |
| Number of skipped attacks:    | 43     |
| Original accuracy:            | 57.0%  |
| Accuracy under attack:        | 57.0%  |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 12.49  |
| Avg num queries:              | 323.93 |
+-------------------------------+--------+


##### attack: kaggle_title + dense_dropout

In [14]:
# Model
# Dataset: kaggle_title
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential()
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2", input_shape=[],
                           dtype=tf.string, trainable=False)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary() 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fit the model
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data=(val_sentences, val_labels))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_4 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_13 (Dense)            (None, 256)               33024     
                                                                 
 dropout_6 (Dropout)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 64)                16448     
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 1)                 65        
                                                                 
Total params: 124,692,225
Trainable params: 49,537
Non-t

In [7]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import PWWSRen2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100  
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mogut\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



textattack: Saving checkpoint under "checkpoints\1670997494184.ta.chkpt" at 2022-12-14 00:58:14 after 20 attacks.


textattack: Saving checkpoint under "checkpoints\1670997495935.ta.chkpt" at 2022-12-14 00:58:15 after 40 attacks.



[Succeeded / Failed / Skipped / Total] 8 / 23 / 13 / 44:  44%|████▍     | 44/100 [00:03<00:04, 11.68it/s]textattack: Saving checkpoint under "checkpoints\1670997497142.ta.chkpt" at 2022-12-14 00:58:17 after 60 attacks.

[Succeeded / Failed / Skipped / Total] 10 / 31 / 23 / 64:  64%|██████▍   | 64/100 [00:04<00:02, 12.92it/s]

textattack: Saving checkpoint under "checkpoints\1670997498047.ta.chkpt" at 2022-12-14 00:58:18 after 80 attacks.


textattack: Saving checkpoint under "checkpoints\1670997500411.ta.chkpt" at 2022-12-14 00:58:20 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 15 / 42 / 43 / 100: 100%|██████████| 100/100 [00:08<00:00, 12.44it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 15     |
| Number of failed attacks:     | 42     |
| Number of skipped attacks:    | 43     |
| Original accuracy:            | 57.0%  |
| Accuracy under attack:        | 42.0%  |
| Attack success rate:          | 26.32% |
| Average perturbed word %:     | 64.14% |
| Average num. words per input: | 13.39  |
| Avg num queries:              | 242.6  |
+-------------------------------+--------+


In [8]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import TextBuggerLi2018 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100  
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



textattack: Saving checkpoint under "checkpoints\1670997537752.ta.chkpt" at 2022-12-14 00:58:57 after 20 attacks.


textattack: Saving checkpoint under "checkpoints\1670997540761.ta.chkpt" at 2022-12-14 00:59:00 after 40 attacks.


textattack: Saving checkpoint under "checkpoints\1670997542689.ta.chkpt" at 2022-12-14 00:59:02 after 60 attacks.


textattack: Saving checkpoint under "checkpoints\1670997544168.ta.chkpt" at 2022-12-14 00:59:04 after 80 attacks.


textattack: Saving checkpoint under "checkpoints\1670997545965.ta.chkpt" at 2022-12-14 00:59:05 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 56 / 43 / 100: 100%|██████████| 100/100 [00:15<00:00,  6.28it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 56     |
| Number of skipped attacks:    | 43     |
| Original accuracy:            | 57.0%  |
| Accuracy under attack:        | 56.0%  |
| Attack success rate:          | 1.75%  |
| Average perturbed word %:     | 8.33%  |
| Average num. words per input: | 13.39  |
| Avg num queries:              | 113.09 |
+-------------------------------+--------+


##### attack: kaggle_title + dense

In [7]:
# Model
# Dataset: kaggle_title
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential()
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2", input_shape=[],
                           dtype=tf.string, trainable=False)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary() 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fit the model
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data=(val_sentences, val_labels))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 128)               124642688 
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 124,651,009
Trainable params: 8,321
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/5
1123/1123 [==============================] - 3s 3ms/step - loss: 0.0387 - accuracy: 0.9931 - val_loss: 0.0111 - val_accuracy: 0.9965
Epoch 2/5
1123/1123 [==============================] - 4s 4ms/step - loss: 0.0100 - accuracy: 0.9971 - val_loss: 0.0086 - val_accuracy: 0.9976
Epoch 3/5
1123

In [8]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import PWWSRen2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100  
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mogut\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 11 / 8 / 20:  20%|██        | 20/100 [00:01<00:07, 10.82it/s]textattack: Saving checkpoint under "checkpoints\1671064583538.ta.chkpt" at 2022-12-14 19:36:23 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 11 / 16 / 29:  29%|██▉       | 29/100 [00:01<00:04, 14.65it/s]

[Succeeded / Failed / Skipped / Total] 2 / 16 / 22 / 40:  40%|████      | 40/100 [00:02<00:03, 16.33it/s]textattack: Saving checkpoint under "checkpoints\1671064584138.ta.chkpt" at 2022-12-14 19:36:24 after 40 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 18 / 23 / 43:  43%|████▎     | 43/100 [00:02<00:03, 16.64it/s]

[Succeeded / Failed / Skipped / Total] 5 / 24 / 31 / 60:  60%|██████    | 60/100 [00:03<00:02, 16.70it/s]textattack: Saving checkpoint under "checkpoints\1671064585280.ta.chkpt" at 2022-12-14 19:36:25 after 60 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 24 / 33 / 63:  63%|██████▎   | 63/100 [00:03<00:02, 16.84it/s]

[Succeeded / Failed / Skipped / Total] 9 / 30 / 41 / 80:  80%|████████  | 80/100 [00:04<00:01, 17.36it/s]textattack: Saving checkpoint under "checkpoints\1671064586296.ta.chkpt" at 2022-12-14 19:36:26 after 80 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 31 / 42 / 82:  82%|████████▏ | 82/100 [00:04<00:01, 17.49it/s]

[Succeeded / Failed / Skipped / Total] 13 / 39 / 48 / 100: 100%|██████████| 100/100 [00:06<00:00, 16.50it/s]textattack: Saving checkpoint under "checkpoints\1671064587750.ta.chkpt" at 2022-12-14 19:36:27 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 39 / 48 / 100: 100%|██████████| 100/100 [00:06<00:00, 16.49it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 39     |
| Number of skipped attacks:    | 48     |
| Original accuracy:            | 52.0%  |
| Accuracy under attack:        | 39.0%  |
| Attack success rate:          | 25.0%  |
| Average perturbed word %:     | 69.47% |
| Average num. words per input: | 13.08  |
| Avg num queries:              | 221.37 |
+-------------------------------+--------+


In [9]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import TextBuggerLi2018 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 11 / 8 / 20:  20%|██        | 20/100 [00:08<00:35,  2.25it/s]textattack: Saving checkpoint under "checkpoints\1671064617793.ta.chkpt" at 2022-12-14 19:36:57 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 12 / 16 / 29:  29%|██▉       | 29/100 [00:09<00:22,  3.21it/s]

[Succeeded / Failed / Skipped / Total] 1 / 17 / 22 / 40:  40%|████      | 40/100 [00:10<00:15,  3.96it/s]textattack: Saving checkpoint under "checkpoints\1671064619008.ta.chkpt" at 2022-12-14 19:36:59 after 40 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 18 / 22 / 41:  41%|████      | 41/100 [00:10<00:14,  4.01it/s]

[Succeeded / Failed / Skipped / Total] 1 / 28 / 31 / 60:  60%|██████    | 60/100 [00:12<00:08,  4.89it/s]textattack: Saving checkpoint under "checkpoints\1671064621166.ta.chkpt" at 2022-12-14 19:37:01 after 60 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 37 / 41 / 80:  80%|████████  | 80/100 [00:13<00:03,  5.77it/s]textattack: Saving checkpoint under "checkpoints\1671064622762.ta.chkpt" at 2022-12-14 19:37:02 after 80 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 38 / 42 / 82:  82%|████████▏ | 82/100 [00:13<00:03,  5.86it/s]

[Succeeded / Failed / Skipped / Total] 2 / 50 / 48 / 100: 100%|██████████| 100/100 [00:16<00:00,  6.18it/s]textattack: Saving checkpoint under "checkpoints\1671064625080.ta.chkpt" at 2022-12-14 19:37:05 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 50 / 48 / 100: 100%|██████████| 100/100 [00:16<00:00,  6.18it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 50     |
| Number of skipped attacks:    | 48     |
| Original accuracy:            | 52.0%  |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 3.85%  |
| Average perturbed word %:     | 8.33%  |
| Average num. words per input: | 13.08  |
| Avg num queries:              | 121.73 |
+-------------------------------+--------+


##### attack: kaggle + dense

In [4]:
# Model
# Dataset: kaggle
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential()
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2", input_shape=[],
                           dtype=tf.string, trainable=False)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary() 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fit the model
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data=(val_sentences, val_labels))

Please fix your imports. Module tensorflow.python.training.tracking.data_structures has been moved to tensorflow.python.trackable.data_structures. The old module will be deleted in version 2.11.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 124,651,009
Trainable params: 8,321
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/5
1123/1123 [==============================] - 8s 7ms/step - loss: 0.1647 - accuracy: 0.9468 - val_loss: 0.1159 - val_accuracy: 0.9639
Epoch 2/5
1123/1123 [==============================] - 9s 8ms/step - loss: 0.1226 - accuracy: 0.9601 - val_loss: 0.1113 - val_accuracy: 0.9646
Epoch 3/5
1123/1

In [7]:
# Model: kaggle, dense, nnlm
from textattack.attack_recipes import PWWSRen2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100  
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mogut\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 9 / 11 / 20:  20%|██        | 20/100 [05:37<22:28, 16.85s/it]textattack: Saving checkpoint under "checkpoints\1671065403063.ta.chkpt" at 2022-12-14 19:50:03 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 9 / 16 / 25:  25%|██▌       | 25/100 [05:37<16:51, 13.48s/it]

[Succeeded / Failed / Skipped / Total] 0 / 19 / 21 / 40:  40%|████      | 40/100 [06:47<10:10, 10.18s/it]textattack: Saving checkpoint under "checkpoints\1671065473203.ta.chkpt" at 2022-12-14 19:51:13 after 40 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 29 / 31 / 60:  60%|██████    | 60/100 [08:06<05:24,  8.10s/it]textattack: Saving checkpoint under "checkpoints\1671065552081.ta.chkpt" at 2022-12-14 19:52:32 after 60 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 34 / 46 / 80:  80%|████████  | 80/100 [10:18<02:34,  7.73s/it]textattack: Saving checkpoint under "checkpoints\1671065684791.ta.chkpt" at 2022-12-14 19:54:44 after 80 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 34 / 51 / 85:  85%|████████▌ | 85/100 [10:18<01:49,  7.28s/it]

[Succeeded / Failed / Skipped / Total] 0 / 45 / 55 / 100: 100%|██████████| 100/100 [12:41<00:00,  7.62s/it]textattack: Saving checkpoint under "checkpoints\1671065827825.ta.chkpt" at 2022-12-14 19:57:07 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 45 / 55 / 100: 100%|██████████| 100/100 [12:41<00:00,  7.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 45     |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 45.0%  |
| Accuracy under attack:        | 45.0%  |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 398.28 |
| Avg num queries:              | 4172.2 |
+-------------------------------+--------+


In [5]:
# Model: kaggle, dense, nnlm
from textattack.attack_recipes import TextBuggerLi2018 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 11 / 9 / 20:  20%|██        | 20/100 [01:41<06:45,  5.07s/it]textattack: Saving checkpoint under "checkpoints\1671066220978.ta.chkpt" at 2022-12-14 20:03:40 after 20 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 18 / 17 / 35:  35%|███▌      | 35/100 [03:02<05:39,  5.23s/it]

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[161878,320] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node EncoderDNN/EmbeddingLookup/EmbeddingLookupUnique/GatherV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_restored_function_body_33533]

##### attack: liar + dense

In [4]:
# Model
# Dataset: kaggle
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential()
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2", input_shape=[],
                           dtype=tf.string, trainable=False)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary() 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fit the model
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data=(val_sentences, val_labels))

Please fix your imports. Module tensorflow.python.training.tracking.data_structures has been moved to tensorflow.python.trackable.data_structures. The old module will be deleted in version 2.11.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 124,651,009
Trainable params: 8,321
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/5
320/320 [==============================] - 2s 3ms/step - loss: 0.6649 - accuracy: 0.5900 - val_loss: 0.6553 - val_accuracy: 0.6117
Epoch 2/5
320/320 [==============================] - 1s 2ms/step - loss: 0.6568 - accuracy: 0.6059 - val_loss: 0.6549 - val_accuracy: 0.6164
Epoch 3/5
320/320 [=

In [5]:
# Model: kaggle, dense, nnlm
from textattack.attack_recipes import PWWSRen2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100  
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mogut\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 14 / 6 / 20:  20%|██        | 20/100 [00:01<00:06, 11.78it/s]textattack: Saving checkpoint under "checkpoints\1671066481571.ta.chkpt" at 2022-12-14 20:08:01 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 16 / 7 / 23:  23%|██▎       | 23/100 [00:01<00:06, 12.34it/s]

[Succeeded / Failed / Skipped / Total] 0 / 27 / 13 / 40:  40%|████      | 40/100 [00:02<00:04, 13.76it/s]textattack: Saving checkpoint under "checkpoints\1671066482783.ta.chkpt" at 2022-12-14 20:08:02 after 40 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 29 / 20 / 49:  50%|█████     | 50/100 [00:03<00:03, 16.03it/s]

[Succeeded / Failed / Skipped / Total] 0 / 34 / 26 / 60:  60%|██████    | 60/100 [00:03<00:02, 16.32it/s]textattack: Saving checkpoint under "checkpoints\1671066483551.ta.chkpt" at 2022-12-14 20:08:03 after 60 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 36 / 27 / 63:  63%|██████▎   | 63/100 [00:03<00:02, 16.32it/s]

[Succeeded / Failed / Skipped / Total] 0 / 44 / 36 / 80:  80%|████████  | 80/100 [00:04<00:01, 16.88it/s]textattack: Saving checkpoint under "checkpoints\1671066484613.ta.chkpt" at 2022-12-14 20:08:04 after 80 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 45 / 36 / 81:  81%|████████  | 81/100 [00:04<00:01, 16.72it/s]

[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [00:06<00:00, 16.37it/s]textattack: Saving checkpoint under "checkpoints\1671066485983.ta.chkpt" at 2022-12-14 20:08:05 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [00:06<00:00, 16.36it/s]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 57     |
| Number of skipped attacks:    | 43     |
| Original accuracy:            | 57.0%  |
| Accuracy under attack:        | 57.0%  |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 17.7   |
| Avg num queries:              | 180.07 |
+-------------------------------+--------+


In [6]:
# Model: kaggle, dense, nnlm
from textattack.attack_recipes import TextBuggerLi2018 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=True
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapHomoglyphSwap
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 14 / 6 / 20:  20%|██        | 20/100 [00:07<00:28,  2.82it/s]textattack: Saving checkpoint under "checkpoints\1671066517527.ta.chkpt" at 2022-12-14 20:08:37 after 20 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 15 / 7 / 22:  22%|██▏       | 22/100 [00:07<00:25,  3.05it/s]

[Succeeded / Failed / Skipped / Total] 0 / 27 / 13 / 40:  40%|████      | 40/100 [00:09<00:13,  4.41it/s]textattack: Saving checkpoint under "checkpoints\1671066519493.ta.chkpt" at 2022-12-14 20:08:39 after 40 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 27 / 14 / 41:  42%|████▏     | 42/100 [00:09<00:12,  4.54it/s]

[Succeeded / Failed / Skipped / Total] 0 / 34 / 26 / 60:  60%|██████    | 60/100 [00:10<00:06,  5.90it/s]textattack: Saving checkpoint under "checkpoints\1671066520602.ta.chkpt" at 2022-12-14 20:08:40 after 60 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 35 / 27 / 62:  62%|██████▏   | 62/100 [00:10<00:06,  6.01it/s]

[Succeeded / Failed / Skipped / Total] 0 / 44 / 36 / 80:  80%|████████  | 80/100 [00:11<00:02,  6.79it/s]textattack: Saving checkpoint under "checkpoints\1671066522205.ta.chkpt" at 2022-12-14 20:08:42 after 80 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 45 / 36 / 81:  81%|████████  | 81/100 [00:11<00:02,  6.80it/s]

[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [00:13<00:00,  7.54it/s]textattack: Saving checkpoint under "checkpoints\1671066523691.ta.chkpt" at 2022-12-14 20:08:43 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 57 / 43 / 100: 100%|██████████| 100/100 [00:13<00:00,  7.54it/s]





+-------------------------------+-------+
| Attack Results                |       |
+-------------------------------+-------+
| Number of successful attacks: | 0     |
| Number of failed attacks:     | 57    |
| Number of skipped attacks:    | 43    |
| Original accuracy:            | 57.0% |
| Accuracy under attack:        | 57.0% |
| Attack success rate:          | 0.0%  |
| Average perturbed word %:     | nan%  |
| Average num. words per input: | 17.7  |
| Avg num queries:              | 66.42 |
+-------------------------------+-------+


##### Show attack detail

In [3]:
# Model
# Dataset: kaggle
import tensorflow_hub as hub
import tensorflow as tf

model = tf.keras.Sequential()
hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2", input_shape=[],
                           dtype=tf.string, trainable=False)
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

model.summary() 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Fit the model
model_history = model.fit(train_sentences,
                          train_labels,
                          epochs=5,
                          validation_data=(val_sentences, val_labels))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 124,651,009
Trainable params: 8,321
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/5
1123/1123 [==============================] - 4s 3ms/step - loss: 0.0387 - accuracy: 0.9926 - val_loss: 0.0092 - val_accuracy: 0.9978
Epoch 2/5
1123/1123 [==============================] - 3s 3ms/step - loss: 0.0105 - accuracy: 0.9967 - val_loss: 0.0079 - val_accuracy: 0.9986
Epoch 3/5
1123/1

In [7]:
# Model: kaggle_title, dense_dropout, nnlm
from textattack.attack_recipes import PWWSRen2019 as attack_recipe

# try first 10 example because quite slow attack speed
i = 100  
train_dataset = []
for example, label in zip(train_sentences[:i], train_labels[:i]):
    train_dataset.append((example, int(label)))
train_dataset = Dataset(train_dataset)

model_wrapper = CustomTensorFlowModelWrapper(model)
attack = attack_recipe.build(model_wrapper)

attack_args = AttackArgs(
    num_examples=i,
    checkpoint_interval=20,
    disable_stdout=False
)

attacker = Attacker(attack, train_dataset, attack_args)
attacker.attack_dataset()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mogut\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 2 / 1 / 3 / 6:   6%|▌         | 6/100 [00:00<00:06, 15.35it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[1 (73%)]] --> [[0 (50%)]]

[[SOFT]] [[COUP]] [[ALERT]]: [[Shocking]] [[Percentage]] Of [[Negative]] [[Coverage]] [[On]] [[Trump]] [[Will]] [[Blow]] Your Mind!

[[subdued]] [[takeover]] [[lively]]: [[traumatise]] [[pct]] Of [[veto]] [[coverage]] [[along]] [[outflank]] [[bequeath]] [[C]] Your Mind!


--------------------------------------------- Result 2 ---------------------------------------------
[[1 (73%)]] --> [[0 (50%)]]

Trump [[Blasts]] Democrats: [[A]] ‘disgrace’ that [[Full]] Cabinet [[Not]] in [[Place]], ‘[[Obstruction]] by Democrats!’

Trump [[blow]] Democrats: [[angstrom]] ‘disgrace’ that [[wide-cut]] Cabinet [[non]] in [[berth]], ‘[[obstacle]] by Democrats!’


--------------------------------------------- Result 3 ---------------------------------------------
[[1 (50%)]] --> [[[SKIPPED]]]

House Republican leader sees Puerto Rico bill before July


------------------------

[Succeeded / Failed / Skipped / Total] 3 / 4 / 5 / 12:  12%|█▏        | 12/100 [00:00<00:04, 18.31it/s]

...

--------------------------------------------- Result 93 ---------------------------------------------
[[1 (73%)]] --> [[[FAILED]]]

TRUMP’S BRILLIANT Director Of Communications OMAROSA Absolutely DESTROYS Joy Behar On The View…And It’s HILARIOUS! [VIDEO]


--------------------------------------------- Result 94 ---------------------------------------------
[[1 (50%)]] --> [[[SKIPPED]]]

Blast hits train in southwestern Pakistan; at least 6 hurt


--------------------------------------------- Result 95 ---------------------------------------------
[[1 (50%)]] --> [[[SKIPPED]]]

Hate speech seeps into U.S. mainstream amid bitter campaign


--------------------------------------------- Result 96 ---------------------------------------------
[[1 (50%)]] --> [[[SKIPPED]]]

Snowden warns of increase in U.S. domestic spying after Trump victory


--------------------------------------------- Result 97 ---------------------------------------------
[[1 (73%)]] --> [[[FAILED]]]

 Alex Jones POSIT

[Succeeded / Failed / Skipped / Total] 20 / 36 / 44 / 100: 100%|██████████| 100/100 [00:05<00:00, 17.51it/s]textattack: Saving checkpoint under "checkpoints\1671077906362.ta.chkpt" at 2022-12-14 23:18:26 after 100 attacks.


--------------------------------------------- Result 100 ---------------------------------------------
[[1 (50%)]] --> [[[SKIPPED]]]

Rohingyas must go home but to safety, Bangladesh says







[Succeeded / Failed / Skipped / Total] 20 / 36 / 44 / 100: 100%|██████████| 100/100 [00:05<00:00, 17.50it/s]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 20     |
| Number of failed attacks:     | 36     |
| Number of skipped attacks:    | 44     |
| Original accuracy:            | 56.0%  |
| Accuracy under attack:        | 36.0%  |
| Attack success rate:          | 35.71% |
| Average perturbed word %:     | 50.54% |
| Average num. words per input: | 13.31  |
| Avg num queries:              | 213.59 |
+-------------------------------+--------+
